In [1]:
!pip install gurobipy

In [2]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# Load data
df = pd.read_excel('dataset.xlsx')
# Remove currency symbol and convert price to float
df['price'] = df['price'].str.replace('£', '').astype(float)
# Exclude products containing "SPF" in their names (case insensitive)
df = df[~df['product_name'].str.contains("SPF", case=False)]

# Create optimization model
model = Model("Maximize_Revenue")

# Define skin types, budget ranges, and products
skins = ['oily', 'normal_dry', 'combination']
budgets = ['under_30', 'under_50', 'under_100']
products = df['product_code'].unique().tolist()

# Add decision variables for selecting products within skin types and budget ranges
decision_variables = model.addVars(products, skins, budgets, vtype=GRB.BINARY, name="decision_variables")

# Percentages of customer base by skin type
skin_type_percentages = {"oily": 0.159, "normal_dry": 0.485, "combination": 0.356}

# Estimated number of customers
estimated_customers = 1774

# Set objective function to maximize total revenue
# Revenue is calculated as the sum of product prices times the number of customers 
# times the percentage of customers with each skin type, for selected products across all skin types and budget ranges
model.setObjective(
    quicksum(
        decision_variables[p, s, b] * df.loc[df['product_code'] == p, 'price'].values[0] * estimated_customers * skin_type_percentages[s]
        for p in products
        for s in skins
        for b in budgets
    ),
    GRB.MAXIMIZE
)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-25


In [4]:
# Required product types for each skin type and budget range
required_products_by_skin_budget = {
    ('normal_dry', 'under_30'): ['Toner', 'Serum'],
    ('combination', 'under_30'): ['Toner', 'Serum'],
    ('oily', 'under_30'): ['Mist', 'Serum'],
    ('normal_dry', 'under_50'): ['Toner', 'Serum', 'Moisturiser'],
    ('combination', 'under_50'): ['Toner', 'Serum', 'Peel'],
    ('oily', 'under_50'): ['Mist', 'Serum', 'Exfoliator'],
    ('normal_dry', 'under_100'): ['Toner', 'Serum', 'Moisturiser', 'Exfoliator', 'Oil'],
    ('combination', 'under_100'): ['Toner', 'Serum', 'Peel', 'Exfoliator', 'Eye Care'],
    ('oily', 'under_100'): ['Mist', 'Serum', 'Exfoliator', 'Peel', 'Eye Care']
}

# Adding constraints for required product types
for (skin, budget), product_types in required_products_by_skin_budget.items():
    for product_type in product_types:
        products_of_type = df[df['product_type'] == product_type]['product_code'].tolist()
        model.addConstr(quicksum(decision_variables[product, skin, budget] for product in products_of_type) >= 1, name=f"constraint_{skin}_{budget}_{product_type}")

In [5]:
# Adding budget constraints
budget_limits = {
    'under_30': 30,
    'under_50': 50,
    'under_100': 100
}

for skin in skins:
    for budget in budgets:
        total_cost = quicksum(df.loc[df['product_code'] == p, 'price'].values[0] * decision_variables[p, skin, budget] for p in products)
        model.addConstr(total_cost <= budget_limits[budget], name=f"Budget_limit_{skin}_{budget}")

In [6]:
# Adding constraints for brand uniqueness
brands = df['brand'].unique().tolist()

for skin in skins:
    for budget in budgets:
        for brand in brands:
            products_of_brand = df[df['brand'] == brand]['product_code'].tolist()
            model.addConstr(quicksum(decision_variables[product, skin, budget] for product in products_of_brand if product in products) <= 1, name=f"Unique_brand_{brand}_{skin}_{budget}")

In [7]:
# Adding constraints for product type uniqueness
product_types = df['product_type'].unique().tolist()

for skin in skins:
    for budget in budgets:
        for product_type in product_types:
            products_of_type = df[df['product_type'] == product_type]['product_code'].tolist()
            model.addConstr(quicksum(decision_variables[product, skin, budget] for product in products_of_type if product in products) <= 1, name=f"Unique_product_type_{product_type}_{skin}_{budget}")


In [8]:
# Ensuring each product is selected at most once across all combinations
for product in products:
    model.addConstr(sum(decision_variables[product, skin, budget] for skin in skins for budget in budgets) <= 1, name=f"Product_{product}_unique_across_all_gifts")


In [9]:
# Solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))



CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2068 rows, 5517 columns and 26962 nonzeros
Model fingerprint: 0xd6de14d3
Variable types: 0 continuous, 5517 integer (5517 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [7e+02, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 311500.95308
Presolve removed 868 rows and 1542 columns
Presolve time: 0.11s
Presolved: 1200 rows, 3975 columns, 23841 nonzeros
Variable types: 0 continuous, 3975 integer (3975 binary)
Found heuristic solution: objective 318507.86280

Root relaxation: objective 3.193200e+05, 1119 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

In [11]:
# print the result
for v in model.getVars():
    if v.x > 0:  
        print(f"{v.varName} = {v.x}")

decision_variables[63,normal_dry,under_100] = 1.0
decision_variables[95,normal_dry,under_50] = 1.0
decision_variables[116,combination,under_100] = 1.0
decision_variables[121,combination,under_50] = 1.0
decision_variables[122,oily,under_30] = 1.0
decision_variables[123,oily,under_50] = 1.0
decision_variables[132,normal_dry,under_30] = 1.0
decision_variables[138,normal_dry,under_50] = 1.0
decision_variables[148,combination,under_30] = 1.0
decision_variables[183,oily,under_100] = 1.0
decision_variables[200,normal_dry,under_100] = 1.0
decision_variables[229,normal_dry,under_100] = 1.0
decision_variables[308,oily,under_30] = 1.0
decision_variables[310,combination,under_50] = 1.0
decision_variables[322,oily,under_50] = 1.0
decision_variables[352,oily,under_100] = 1.0
decision_variables[569,normal_dry,under_100] = 1.0
decision_variables[573,oily,under_30] = 1.0
decision_variables[575,combination,under_100] = 1.0
decision_variables[584,combination,under_50] = 1.0
decision_variables[595,oily,un

In [12]:
# Initialize an empty DataFrame as a matrix
matrix_df = pd.DataFrame(index=skins, columns=budgets, dtype='object')

# Initialize an empty list for each cell in the DataFrame
for skin in skins:
    for budget in budgets:
        matrix_df.at[skin, budget] = []

# Iterate through decision variables, if a product is selected, then add it to the respective cell
for v in model.getVars():
    if v.x > 0:  # If the product is selected
        # Parse product_code, skin, budget from the variable name
        details = v.varName.split('[')[1].split(']')[0].split(',')
        product_code = details[0]
        skin = details[1].strip().strip('"')
        budget = details[2].strip().strip('"')
        
        # Add product_code to the corresponding cell
        matrix_df.at[skin, budget].append(product_code)

# Convert the list in each cell to a string for easy printing and viewing
for skin in skins:
    for budget in budgets:
        matrix_df.at[skin, budget] = ', '.join(matrix_df.at[skin, budget])

# Print the result
print(matrix_df)



                       under_30            under_50  \
oily         122, 308, 573, 679       123, 322, 917   
normal_dry             132, 819        95, 138, 850   
combination            148, 823  121, 310, 584, 821   

                               under_100  
oily             183, 352, 595, 613, 938  
normal_dry   63, 200, 229, 569, 830, 925  
combination      116, 575, 636, 840, 923  


In [19]:
total_revenue = 0

for v in model.getVars():
    if v.x > 0:  # Only consider selected products
        # Parse product_code, skin_type from the variable name
        parts = v.varName.split('[')[1].split(']')[0].split(',')
        product_code = parts[0].strip().strip('"')
        skin_type = parts[1].strip().strip('"')
        
        # Find the corresponding price
        matched_prices = df.loc[df['product_code'].astype(str) == str(product_code), 'price'].values
        
        if len(matched_prices) > 0:
            price = matched_prices[0]
            # Calculate the contribution of this product to total revenue
            revenue_contribution = price * v.x * estimated_customers * skin_type_percentages[skin_type]
            total_revenue += revenue_contribution
        else:
            print(f"Price not found for product_code: {product_code}")

print(f"Total Revenue: £{total_revenue:.2f}")





Total Revenue: £319320.00
